In [2]:
# %%
import pandas as pd
import json
import json
import os


In [ ]:

# read in json from different folder
with open("../results/skills_gap_analysis/job_gpt-3.5-turbo_2sent_n1500_jBd_detailed.json", "r") as f:
    jobs = json.load(f)
with open("../results/skills_gap_analysis/course_gpt-3.5-turbo_2sent_n500_jBd_V4_detailed.json", "r") as f:
    courses = json.load(f)

# read in taxonomy from different folder
taxonomy = pd.read_csv("../../data/taxonomy/taxonomy_V4.csv")
keep_cols = [
    "unique_id",
    "Type Level 1",
    "Type Level 2",
    "Type Level 3",
    "Type Level 4",
]
taxonomy = taxonomy[keep_cols]

courses = {course_id:courses[course_id]['to_acquire'] for course_id in courses.keys()}

results_dict = {'jobs': jobs, 'courses': courses}

In [19]:

for data_type, results in results_dict.items():
    print(data_type, len(results))
    matched_ids = {}
    for job_id in results:
        job = results[job_id]
        for sent in job:
            matched = sent["matched_skills"]
            for skill, data in matched.items():
                if data["unique_id"] not in matched_ids:
                    matched_ids[data["unique_id"]] = 1
                else:
                    matched_ids[data["unique_id"]] += 1
                # matched_ids.append(data["unique_id"])


    taxonomy["matched_count_"+data_type] = taxonomy["unique_id"].map(matched_ids).fillna(0)

jobs 1147
courses 124


In [55]:
for data_type in results_dict.keys():
    lv1 = (
        taxonomy.groupby("Type Level 1")
        .sum()
        .sort_values(by="matched_count_jobs", ascending=False)
        .drop(columns="unique_id")
    )
    lv2 = (
        taxonomy.groupby(["Type Level 1", "Type Level 2"])
        .sum()
        .sort_values(by="matched_count_jobs", ascending=False)
        .drop(columns="unique_id")
    )
    lv3 = (
        taxonomy.groupby(["Type Level 1", "Type Level 2", "Type Level 3"])
        .sum()
        .sort_values(by="matched_count_jobs", ascending=False)
        .drop(columns="unique_id")
    )
    lv4 = (
        taxonomy.groupby(["Type Level 1", "Type Level 2", "Type Level 3", "Type Level 4"])
        .sum()
        .sort_values(by="matched_count_jobs", ascending=False)
        .drop(columns="unique_id")
    )


/tmp/ipykernel_141383/1297889188.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()
/tmp/ipykernel_141383/1297889188.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()
/tmp/ipykernel_141383/1297889188.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()
/tmp/ipykernel_141383/1297889188.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version

In [65]:
# lv1
# lv2
# lv3
lv2

matched_count_jobs  \
Type Level 1              Type Level 2                                               
Fachkompetenz             Informationssicherheit                           12000.0   
                          Management der ICT                               11505.0   
                          Projektmanagement                                 3564.0   
                          Network Operation                                 2025.0   
                          Service Design                                    1690.0   
                          Application Engineering                           1638.0   
Kompetenzelemente         Soziale Fertigkeit                                1520.0   
Fachkompetenz             ICT Service Operation                             1342.0   
                          Application Operation                              948.0   
Kompetenzelemente         Leistungsorientierung                              915.0   
Fachkompetenz             Service Engineering                                552.0   
                          Ergonomie, Usability                               336.0   
                          Betriebswirtschaft der ICT                         330.0   
Kompetenzelemente         Gewissenhaftigkeit                                 268.0   
                          Kognitive Fertigkeit                               232.0   
Fachkompetenz             Gebäudeautomation                                  160.0   
                          Hardware Operation                                 108.0   
                          Data Engineering/Data Management                   105.0   
                          Organisation                                       105.0   
Kompetenzelemente         Beziehungsorientierung                              95.0   
Sozial- & Selbstkompetenz Kollaboration                                       45.0   
Kompetenzelemente         Flexibilität                                        19.0   
Sozial- & Selbstkompetenz Kommunikation                                       15.0   
Fachkompetenz             Technical Software Engineering                      15.0   
Kompetenzelemente         Physische Fertigkeit                                14.0   
                          Psychomotorische Fertigkeit                         12.0   
Sozial- & Selbstkompetenz Selbsttransformation                                12.0   
Fachkompetenz             Qualitätsmanagement                                 10.0   
Kompetenzelemente         Unabhängigkeit                                       7.0   
Fachkompetenz             Hardware Engineering                                 3.0   
                          Network Engineering                                  2.0   
Kompetenzelemente         Toleranz                                             2.0   
Sozial- & Selbstkompetenz Selbstorganisation                                   1.0   
Fachkompetenz             Business- & Requirements-Engineering                 0.0   
Kompetenzelemente         Psychische Fertigkeit                                0.0   
Sozial- & Selbstkompetenz Führung                                              0.0   
Fachkompetenz             Architekturen                                        0.0   
Sozial- & Selbstkompetenz Selbstreflexion                                      0.0   
Fachkompetenz             Risikomanagement                                     0.0   

                                                                matched_count_courses  
Type Level 1              Type Level 2                                                 
Fachkompetenz             Informationssicherheit                                864.0  
                          Management der ICT                                    450.0  
                          Projektmanagement                                      72.0  
                          Network Operation                                      75.0  
         

In [38]:
# Cases where we have gaps, level by level:
lv2[(lv2["matched_count_jobs"] > 10) & (lv2["matched_count_courses"] == 0)]


matched_count_jobs  \
Type Level 1      Type Level 2                                         
Fachkompetenz     Application Operation                        948.0   
                  Gebäudeautomation                            160.0   
                  Hardware Operation                           108.0   
Kompetenzelemente Flexibilität                                  19.0   
Fachkompetenz     Technical Software Engineering                15.0   

                                                  matched_count_courses  
Type Level 1      Type Level 2                                           
Fachkompetenz     Application Operation                             0.0  
                  Gebäudeautomation                                 0.0  
                  Hardware Operation                                0.0  
Kompetenzelemente Flexibilität                                      0.0  
Fachkompetenz     Technical Software Engineering                    0.0

In [39]:
lv3[(lv3["matched_count_jobs"] > 10) & (lv3["matched_count_courses"] == 0)]


,,matched_count_jobs,matched_count_courses
Type Level 2,Type Level 3,,
Beziehungsorientierung,Teamorientierung,84.0,0.0
Leistungsorientierung,Tatkraft,31.0,0.0
Kognitive Fertigkeit,Sprachliche Fertigkeiten,12.0,0.0
Psychomotorische Fertigkeit,Feinmotorik,11.0,0.0


In [40]:
lv4[(lv4["matched_count_jobs"] > 10) & (lv4["matched_count_courses"] == 0)]


,,,matched_count_jobs,matched_count_courses
Type Level 2,Type Level 3,Type Level 4,,
Kognitive Fertigkeit,Wahrnehmungsfähigkeiten,Aufnahmefähigkeit,13.0,0.0


# focus on rollouter job

In [27]:
rollout_ids = json.load(open('list_rollouter_job_ids.json'))
rollout_ids = [str(rollout_id) for rollout_id in rollout_ids]
with open("../results/rollouter_jobs/job_gpt-3.5-turbo_ids_2sent_n10_jBd_V4_detailed.json", "r") as f:
    jobs = json.load(f)
taxonomy = pd.read_csv("../../data/taxonomy/taxonomy_V4.csv")
keep_cols = [
    "unique_id",
    "Type Level 1",
    "Type Level 2",
    "Type Level 3",
    "Type Level 4",
]
taxonomy = taxonomy[keep_cols]

jobs_rollout = {job_id:job_extract for job_id, job_extract in jobs.items() if int(job_id) in rollout_ids}
print('Jobs etxracted for Rollouter:', len(jobs_rollout), 'out of', len(rollout_ids))


matched_ids = {}
for job in jobs_rollout.values():
    for sent in job:
        matched = sent["matched_skills"]
        for skill, data in matched.items():
            if data["unique_id"] not in matched_ids:
                matched_ids[data["unique_id"]] = 1
            else:
                matched_ids[data["unique_id"]] += 1


taxonomy["matched_count_rollouter"] = taxonomy["unique_id"].map(matched_ids).fillna(0)

FileNotFoundError: [Errno 2] No such file or directory: '../results/rollouter_jobs/job_gpt-3.5-turbo_ids_2sent_n10_jBd_V4_detailed.json.json'

In [20]:
rollout_ids.append('14419')

In [23]:
[id for id in rollout_ids if str(id) in jobs.keys()]

[]

In [25]:
jobs.keys()

dict_keys(['14419', '14421', '14437', '14422', '14423', '14433', '14435', '14468', '14470', '14442', '14473', '14479', '4814', '14481', '14483', '14485', '14887', '14477', '14478', '14488', '14495', '13129', '14486', '14490', '14496', '14492', '14499', '14508', '14509', '14516', '14503', '14517', '14515', '14514', '5305', '14524', '14526', '5453', '14519', '14520', '14527', '14533', '13422', '14534', '14535', '14536', '14539', '14543', '14545', '14620', '13130', '10986', '14541', '14542', '14615', '13135', '14466', '14627', '7746', '14476', '14629', '14632', '14642', '10860', '10687', '14646', '8300', '8310', '13178', '14648', '13182', '13319', '8367', '8487', '8488', '13415', '10692', '8685', '8785', '10965', '11000', '10764', '8957', '9152', '13416', '13423', '13429', '11055', '13175', '9754', '9155', '16488', '13504', '11058', '16494', '11061', '16497', '3063', '16499', '10109', '10920', '13506', '13526', '11640', '11063', '11642', '10376', '16500', '3765', '13579', '10579', '13594'